In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

N = 100
x = np.random.default_rng().uniform(0.0, 1.0, N)
y = np.random.default_rng().uniform(0.0, 1.0, N)

## Заголовок

Тест

$ \lim\limits_{x \to \infty} x^e = 0 $